In [20]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import importlib
import plotly.express as px

#Change me!
start = datetime.datetime.today() + relativedelta.relativedelta(years=-10)
split_unit = "1Y"
modulename = "BacktestingStrategies.Strategy_MacdCross"
classname = "btst.MACDCross"
end = datetime.date.today()

data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = data.astype("double")

initial = 1000_000
fname = f"html/plot-"

In [21]:
#日付データを検証期間単位ごとにSplit
#https://kakakakakku.hatenablog.com/entry/2021/05/24/002542
grouped_data = data.groupby(pd.Grouper(freq = split_unit))
dfs = [group for _, group in grouped_data]

In [22]:
#検証期間単位ごとにバックテスト＆結果を蓄積
#親ディレクトリ経由で呼び出し   # https://qiita.com/yokohama4580/items/466a483ae022d264c8ee
import os
import sys
sys.path.append(os.pardir)

btst = importlib.import_module(modulename)
# from BacktestingStrategies import Strategy_RsiOscillator as btst
importlib.reload(btst)
import warnings
warnings.simplefilter('ignore')

returns = []

for monthly_data in dfs:
    # バックテストを設定
    bt = Backtest(
        monthly_data, # チャートデータ
        eval(classname), # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    returns.append(output)

In [23]:
import statistics

df = pd.DataFrame(returns)

#最低と最大の期間のBacktestingを表示＆実行
lines = "１期間単位: " + split_unit + "\n"
lines += "テスト期間: " + str(len(returns)) + "\n"
res = round(statistics.mean(df["Return [%]"]), 4)

#リターンが最小のものの期間を取得
min_return = df[df["# Trades"] != 0]["Return [%]"].min()
min_period_start = pd.to_datetime(df[df["Return [%]"] == min_return]["Start"]).values[0]
min_period_end = pd.to_datetime(df[df["Return [%]"] == min_return]["End"]).values[0]
df_min = data[min_period_start : min_period_end]

#リターンが2番目、3番目に小さいものの期間を取得
min_return_3 = df[df["# Trades"] != 0]["Return [%]"].nsmallest(3)
min2_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["Start"]).values[0]
min2_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["End"]).values[0]
min3_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["Start"]).values[0]
min3_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["End"]).values[0]
df_min2 = data[min2_period_start : min2_period_end]
df_min3 = data[min3_period_start : min3_period_end]

#リターンが最大のものの期間を取得
max_return = df[df["# Trades"] != 0]["Return [%]"].max()
max_period_start = pd.to_datetime(df[df["Return [%]"] == max_return]["Start"]).values[0]
max_period_end = pd.to_datetime(df[df["Return [%]"] == max_return]["End"]).values[0]
df_max = data[max_period_start : max_period_end]

lines += "平均Return [%]: " + str(res) + "\n"
lines += "最大Return [%]: " + str(max_return) + "\n"
lines += "最大Return期間: " + str(max_period_start) + " - " + str(max_period_end) + "\n"
lines += "最低Return [%]: " + str(min_return) + "\n"
lines += "最低Return期間: " + str(min_period_start) + " - " + str(min_period_end) 

print(lines)



１期間単位: 1Y
テスト期間: 11
平均Return [%]: 8.8137
最大Return [%]: 23.332619999999995
最大Return期間: 2013-01-04T00:00:00.000000000 - 2013-12-30T00:00:00.000000000
最低Return [%]: -13.627303849999992
最低Return期間: 2018-01-01T00:00:00.000000000 - 2018-12-31T00:00:00.000000000


In [24]:
#取引ない月込み
fig = px.box(df["Return [%]"], points = "all")
fig.update_layout(
    xaxis_title = "Strategy",
    yaxis_title = "Return [%]"
)
fig.show()

In [25]:
#最低と最高の期間のバックテストを表示

print("----------WORST----------")
bt = Backtest(
    df_min, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min.html") # 実行結果（グラフ）

print("----------MIN2----------")
bt = Backtest(
    df_min2, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min2.html") # 実行結果（グラフ）

print("----------MIN3----------")
bt = Backtest(
    df_min3, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min3.html") # 実行結果（グラフ）

print("----------BEST----------")
bt = Backtest(
    df_max, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_max.html") # 実行結果（グラフ）

----------WORST----------
Start                     2018-01-01 00:00:00
End                       2018-12-31 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   19.157088
Equity Final [$]                  863726.9615
Equity Peak [$]                   1050265.436
Return [%]                         -13.627304
Buy & Hold Return [%]               -24.01537
Return (Ann.) [%]                  -13.189873
Volatility (Ann.) [%]               11.222726
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -18.532313
Avg. Drawdown [%]                  -18.532313
Max. Drawdown Duration      307 days 00:00:00
Avg. Drawdown Duration      307 days 00:00:00
# Trades                                    6
Win Rate [%]                             50.0
Best Trade [%]                       3.529039
Worst Trade [%]                     -7.404266
Avg. Tra

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,52,39,45,18953.3570,17550.0,-72974.5640,-0.074043,2018-02-23,2018-03-05,10 days
1,48,50,56,19124.1985,17940.0,-56841.5280,-0.061921,2018-03-12,2018-03-20,8 days
2,47,64,74,18400.6345,19050.0,30520.1785,0.035290,2018-03-30,2018-04-13,14 days
3,44,139,146,20149.2475,20270.0,5313.1100,0.005993,2018-07-13,2018-07-24,11 days
4,45,222,228,19727.1685,18700.0,-46222.5825,-0.052069,2018-11-07,2018-11-15,8 days
5,45,234,243,18802.6145,18890.0,3932.3475,0.004648,2018-11-23,2018-12-06,13 days


----------MIN2----------
Start                     2015-01-05 00:00:00
End                       2015-12-30 00:00:00
Duration                    359 days 00:00:00
Exposure Time [%]                    6.967213
Equity Final [$]                   965268.179
Equity Peak [$]                   1036548.179
Return [%]                          -3.473182
Buy & Hold Return [%]               15.443425
Return (Ann.) [%]                   -3.584991
Volatility (Ann.) [%]               11.695372
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.846495
Avg. Drawdown [%]                   -5.664101
Max. Drawdown Duration       82 days 00:00:00
Avg. Drawdown Duration       28 days 00:00:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                       2.111944
Worst Trade [%]                     -5.506776
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,70,173,180,14159.7455,13380.0,-54582.185,-0.055068,2015-09-14,2015-09-28,14 days
1,72,184,192,13054.3005,13330.0,19850.364,0.021119,2015-10-02,2015-10-15,13 days


----------MIN3----------
Start                     2021-01-04 00:00:00
End                       2021-12-30 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   13.877551
Equity Final [$]                1021110.60025
Equity Peak [$]                 1075380.60025
Return [%]                            2.11106
Buy & Hold Return [%]               10.052724
Return (Ann.) [%]                    2.172026
Volatility (Ann.) [%]               15.017126
Sharpe Ratio                         0.144637
Sortino Ratio                         0.20504
Calmar Ratio                         0.154104
Max. Drawdown [%]                  -14.094528
Avg. Drawdown [%]                   -6.430815
Max. Drawdown Duration      120 days 00:00:00
Avg. Drawdown Duration       58 days 00:00:00
# Trades                                    4
Win Rate [%]                             25.0
Best Trade [%]                      19.408926
Worst Trade [%]                     -9.859399
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,60,84,87,16440.98200,14820.0,-97258.92000,-0.098594,2021-05-11,2021-05-14,3 days
1,61,145,152,14652.17100,14180.0,-28802.43100,-0.032225,2021-08-06,2021-08-18,12 days
2,59,157,172,14672.27000,17520.0,168016.07000,0.194089,2021-08-25,2021-09-15,21 days
3,67,232,237,15451.10625,15140.0,-20844.11875,-0.020135,2021-12-14,2021-12-21,7 days


----------BEST----------
Start                     2013-01-04 00:00:00
End                       2013-12-30 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                    9.795918
Equity Final [$]                    1233326.2
Equity Peak [$]                     1233326.2
Return [%]                           23.33262
Buy & Hold Return [%]              118.181818
Return (Ann.) [%]                   24.073829
Volatility (Ann.) [%]               21.009893
Sharpe Ratio                         1.145833
Sortino Ratio                        2.678879
Calmar Ratio                         2.357993
Max. Drawdown [%]                  -10.209456
Avg. Drawdown [%]                   -3.210853
Max. Drawdown Duration        7 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                      14.481858
Worst Trade [%]                      7.838293
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,120,115,129,8280.788,9480.0,143905.44,0.144819,2013-06-24,2013-07-12,18 days
1,132,166,174,8642.570,9320.0,89420.76,0.078383,2013-09-04,2013-09-17,13 days


Row(id='20094', ...)